In [ ]:
import os
os.chdir(r"Q:/sachuriga/Sachuriga_Python/quattrocolo-nwb4fp/src")
import pandas as pd
from neurochat.nc_data import NData
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
import neurochat.nc_plot as nc_plot
from neurochat.nc_lfp import NLfp
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import math
import pynapple as nap
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import sys
import nwb4fp.analyses.maps as mapp
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap_ax,plot_path
from nwb4fp.analyses.fields import separate_fields_by_laplace, separate_fields_by_dilation,find_peaks,separate_fields_by_laplace_of_gaussian,calculate_field_centers,distance_to_edge_function, remove_fields_by_area, map_pass_to_unit_circle,which_field,compute_crossings
from elephant.statistics import time_histogram, instantaneous_rate
from nwb4fp.analyses import maps
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,unit_location_ch,calculate_spatial_coherence,calculate_spatial_stability,coherence, find_run_indices
from scipy.ndimage import gaussian_filter
import ast
import pandas as pd
import numpy as np
from scipy import stats
import numpy as np
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [ ]:
import pandas as pd
import os
## add new measurements to pkl
df_loaded_cell_type = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

In [ ]:
## Speed score collections

u_files = np.unique(df_loaded_cell_type['session_id'])
base_folder = r"S:\Sachuriga\nwb\test4neo"

speed_scores = []
FR = []
shuffle_speed_scores = []
speed_collect = []
speed_t = []

for file in u_files:
    temp_nwb =  nap.load_file(fr'{base_folder}/{file}')
    pkl = df_loaded_cell_type[df_loaded_cell_type['session_id']==file].copy().reset_index(drop=True)
    units_data = temp_nwb['units']
    units = units_data[units_data['unit_name'].isin(pkl['unit_name'])]
    pos_cord = load_speed_fromNWB(temp_nwb['XY_mid_brain'])
    t=pos_cord[:,0]
    ## filter speed
    raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(pos_cord[:,0], # times
                                pos_cord[:,1], # x
                                pos_cord[:,2], # y
                                filter_speed=True, 
                                min_speed = 0.05)
    dt = np.diff(t)
    # Use scipy.stats.mode (scipy < 1.9)
    mode_val = round(stats.mode(dt, keepdims=True).mode[0],2)

    for i in units.index:
        speed_t.append(t)

        # --- Example data ---
        # Simulated spike times (in seconds)
        spike_times = units[i].index.values  # 500 spikes over 10 seconds

        # Simulated frame times (e.g., 30 Hz video = frame every ~33.3 ms)
        frame_rate = 1 / mode_val # Hz
        frame_times = t  # From 0 to 10 seconds

        # --- 1. Bin spike times into spike counts per frame interval ---
        spike_counts, _ = np.histogram(spike_times, bins=frame_times)

        # --- 2. Estimate dt (inter-frame interval) ---
        dt = mode_val  # Should be ~1 / frame_rate

        # --- 3. Create Gaussian smoothing kernel with sigma = 250 ms ---
        sigma_ms = 100  # in milliseconds
        sigma_samples = sigma_ms / 1000 / dt  # convert ms to samples

        # --- 4. Smooth the spike counts to get firing rate (spikes/s) ---
        firing_rate = gaussian_filter1d(spike_counts / dt, sigma=sigma_samples)

        # --- Optional: plot the result ---
        time_axis = frame_times[:-1] + dt / 2  # center of each bin
        speed = smoothed_speed[:len(firing_rate)]  # make sure dimensions match

        # --- Compute Pearson correlation ---
        valid = ~np.isnan(firing_rate) & ~np.isnan(speed)
        r, pval = pearsonr(firing_rate[valid], speed[valid])
        speed_scores.append(r)
        speed_collect.append(smoothed_speed)
        FR.append(firing_rate)
        temp_shuffle = []
        shift_range=90

        for ii in range(100):
            # Random circular shift
            max_shift_bins = int(shift_range / mode_val)
            shift = np.random.randint(-max_shift_bins, max_shift_bins + 1)
            firing_rate_shifted = np.roll(firing_rate, shift)
            r_shuffle, pval = pearsonr(firing_rate_shifted[valid], speed[valid])
            temp_shuffle.append(r_shuffle)
        shuffle_speed_scores.append(temp_shuffle)

df_loaded_cell_type['speed_scores']=None
df_loaded_cell_type['speed_collect']=None
df_loaded_cell_type['speed_fr']=None
df_loaded_cell_type['shuffle_speed_scores']=None
df_loaded_cell_type['speed']=None
df_loaded_cell_type['speed_t'] = None

df_loaded_cell_type['speed_t'] = speed_t
df_loaded_cell_type['speed_scores']=speed_scores
df_loaded_cell_type['speed_collect']=speed_collect
df_loaded_cell_type['speed_fr']=FR
df_loaded_cell_type['shuffle_speed_scores']=shuffle_speed_scores
df_loaded_cell_type.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

In [ ]:
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')

df_int=None
df_py=None
units=df_loaded.reset_index(drop=True)
scores=[]


for i in units.index:
    
    t = units['speed_t'].loc[i]
    dt= np.diff(t)
    mode_val = round(stats.mode(dt, keepdims=True).mode[0],2)
    # --- Example data ---
    # Simulated spike times (in seconds)
    spike_times = units['spike_times'].loc[i]  # 500 spikes over 10 seconds

    # Simulated frame times (e.g., 30 Hz video = frame every ~33.3 ms)
    frame_rate = 1 / mode_val # Hz
    frame_times =  units['speed_t'].loc[i]  # From 0 to 10 seconds

    # --- 1. Bin spike times into spike counts per frame interval ---
    spike_counts, _ = np.histogram(spike_times, bins=frame_times)

    # --- 2. Estimate dt (inter-frame interval) ---
    dt = mode_val  # Should be ~1 / frame_rate

    # --- 3. Create Gaussian smoothing kernel with sigma = 250 ms ---
    sigma_ms = 250  # in milliseconds
    sigma_samples = sigma_ms / 1000 / dt  # convert ms to samples

    # --- 4. Smooth the spike counts to get firing rate (spikes/s) ---
    firing_rate = gaussian_filter1d(spike_counts / dt, sigma=sigma_samples)
    smoothed_speed = units['speed_collect'].loc[i]
    # --- Optional: plot the result ---
    time_axis = frame_times[:-1] + dt / 2  # center of each bin
    speed = smoothed_speed[:len(firing_rate)]  # make sure dimensions match

    # --- Compute Pearson correlation ---
    valid = ~np.isnan(firing_rate) & ~np.isnan(speed)
    r, pval = pearsonr(firing_rate[valid], speed[valid])
    print(r)
    scores.append(r)

df_good = units
units['speed_score']=None
units['speed_score']=scores
df_int = df_good[(df_good['buzaki_py_cell_type'] == "narrow_spike_interneurons") & (df_good['session'] == "A")]
df_py = df_good[(df_good['buzaki_py_cell_type'] == "pyramidal") & (df_good['session'] == "A")]

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


scores=[]
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
units=df_loaded.reset_index(drop=True)
fr = []
for i in units.index:

    t = units['speed_t'].loc[i]
    dt= np.diff(t)
    mode_val = round(stats.mode(dt, keepdims=True).mode[0],2)
    # --- Example data ---
    # Simulated spike times (in seconds)
    spike_times = units['spike_times'].loc[i]  # 500 spikes over 10 seconds

    # Simulated frame times (e.g., 30 Hz video = frame every ~33.3 ms)
    frame_rate = 1 / mode_val # Hz
    frame_times =  units['speed_t'].loc[i]  # From 0 to 10 seconds

    # --- 1. Bin spike times into spike counts per frame interval ---
    spike_counts, _ = np.histogram(spike_times, bins=frame_times)

    # --- 2. Estimate dt (inter-frame interval) ---
    dt = mode_val  # Should be ~1 / frame_rate

    # --- 3. Create Gaussian smoothing kernel with sigma = 250 ms ---
    sigma_ms = 250  # in milliseconds
    sigma_samples = sigma_ms / 1000 / dt  # convert ms to samples

    # --- 4. Smooth the spike counts to get firing rate (spikes/s) ---
    firing_rate = gaussian_filter1d(spike_counts / dt, sigma=sigma_samples)
    smoothed_speed = units['speed_collect'].loc[i]
    # --- Optional: plot the result ---
    time_axis = frame_times[:-1] + dt / 2  # center of each bin
    speed = smoothed_speed[:len(firing_rate)]  # make sure dimensions match

    # --- Compute Pearson correlation ---
    valid = ~np.isnan(firing_rate) & ~np.isnan(speed)
    r, pval = pearsonr(firing_rate[valid], speed[valid])
    print(r)
    scores.append(r)
    fr.append(firing_rate)
units['speed_score']=None
units['speed_score']=scores

units['speed_fr']=None
units['speed_fr']=fr


df_good=units
df_good=df_good[df_good['session'] == "A"]
df_int = df_good[(df_good['buzaki_py_cell_type'] == "narrow_spike_interneurons") & (df_good['session'] == "A")]
df_py = df_good[(df_good['buzaki_py_cell_type'] == "pyramidal") & (df_good['session'] == "A")]

control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
control_df = df_int[df_int['animal_id'].isin(control_ids)]
exp_df = df_int[df_int['animal_id'].isin(exp_ids)]
top_10_indices_exp = exp_df['speed_score'].nlargest(12).index.values
top_10_indices_control = control_df['speed_score'].nlargest(12).index.values
units.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')


In [ ]:
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_a  = df_good[(df_good['buzaki_cell_type']=='narrow_spike_interneurons') & (df_good['session'] == "A")]

base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']

# Separate into control and experimental groups
control_df = df_a[df_a['animal_id'].isin(control_ids)]
exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

In [ ]:
    [round(78/113,2)*100, round((113-78)/113,2)*100],  # Group 1: Dark blue (71.4%), Light blue (28.6%)
    [round(64/94,2)*100, round((94-64)/94,2)*100],  # Group 2: Dark blue (71.1%), Light blue (28.9%)

In [ ]:
round(64/94,2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats  # For Gaussian smoothing (placeholder)
from numpy import quantile  # For quantile calculation
import seaborn as sns

fig = plt.figure(figsize=(7.2, 8), dpi=2400)
plt.rcParams.update({'font.size': 6})
gs = gridspec.GridSpec(6, 6, height_ratios=[.8, .8, .8, .8, 1, 1], width_ratios=[0.8, 0.8, 0.8, 0.8, 0.8, 0.8])  # First row taller
plt.rcParams.update({
    'axes.labelpad': -0.1,
    'ytick.major.pad': -0.1,
    'xtick.major.pad': -0.1,
    'ytick.major.size': 2,
    'xtick.major.size': 2
})

ax1_1 = fig.add_subplot(gs[0, 0])
ax1_2 = fig.add_subplot(gs[0, 1])
ax1_3 = fig.add_subplot(gs[0, 2])
ax1_4 = fig.add_subplot(gs[0, 3])
ax1_5 = fig.add_subplot(gs[0, 4])
ax1_6 = fig.add_subplot(gs[0, 5])

ax2_1 = fig.add_subplot(gs[1, 0])
ax2_2 = fig.add_subplot(gs[1, 1])
ax2_3 = fig.add_subplot(gs[1, 2])
ax2_4 = fig.add_subplot(gs[1, 3])
ax2_5 = fig.add_subplot(gs[1, 4])
ax2_6 = fig.add_subplot(gs[1, 5])

ax3_1 = fig.add_subplot(gs[2, 0])
ax3_2 = fig.add_subplot(gs[2, 1])
ax3_3 = fig.add_subplot(gs[2, 2])
ax3_4 = fig.add_subplot(gs[2, 3])
ax3_5 = fig.add_subplot(gs[2, 4])
ax3_6 = fig.add_subplot(gs[2, 5])

ax4_1 = fig.add_subplot(gs[3, 0])
ax4_2 = fig.add_subplot(gs[3, 1])
ax4_3 = fig.add_subplot(gs[3, 2])
ax4_4 = fig.add_subplot(gs[3, 3])
ax4_5 = fig.add_subplot(gs[3, 4])
ax4_6 = fig.add_subplot(gs[3, 5])


ax5_1 = fig.add_subplot(gs[4, 0])
flattened_list = [item for sublist in df_int['shuffle_speed_scores'] for item in sublist]
counts, bin_edges = np.histogram(flattened_list, bins=200,density=True)
fractions = counts / np.sum(counts)
ax5_1.stairs(counts , bin_edges, color='red')

counts_real, bin_edges_real = np.histogram(df_good['speed_score'], bins=100,density=True)
fractions_real = counts_real / np.sum(counts_real)
ax5_1.stairs(counts_real , bin_edges_real, color='black')
ax5_1.spines['top'].set_visible(False)
ax5_1.spines['right'].set_visible(False)
ax5_1.spines['bottom'].set_visible(True)
ax5_1.spines['left'].set_visible(True)
y_max = ax5_1.get_ylim()[1]
bar_height = y_max * 0.1
ax5_1.text(-0.6, y_max * 1.1, f'Shuffled data', color='red',fontsize=4)
ax5_1.text(-0.6, y_max * 1.05, f'All units from CA1', color='black',fontsize=4)
ax5_1.axvline(x=0.3, color='red', linestyle='--', linewidth=1)
ax5_1.axvline(x=-0.3, color='red', linestyle='--', linewidth=1)
ax5_1.set_xlabel('Speed score')
ax5_1.set_ylabel('Density')


ax5_2 = fig.add_subplot(gs[4, 1])

counts, bin_edges = np.histogram(df_int['speed_score'], bins=50,density=True)
fractions = counts / np.sum(counts)
ax5_2.stairs(counts , bin_edges, color='red')

counts_real, bin_edges_real = np.histogram(df_py['speed_score'], bins=50,density=True)
fractions_real = counts_real / np.sum(counts_real)
ax5_2.stairs(counts_real , bin_edges_real, color='black')
ax5_2.spines['top'].set_visible(False)
ax5_2.spines['right'].set_visible(False)
ax5_2.spines['bottom'].set_visible(True)
ax5_2.spines['left'].set_visible(True)
y_max = ax5_2.get_ylim()[1]
bar_height = y_max * 0.1
ax5_2.text(-0.6, y_max * 1.1, f'Interneurons', color='red',fontsize=4)
ax5_2.text(-0.6, y_max * 1.05, f'Pyramidal neurons', color='black',fontsize=4)
ax5_2.axvline(x=0.3, color='red', linestyle='--', linewidth=1)
ax5_2.axvline(x=-0.3, color='red', linestyle='--', linewidth=1)
ax5_2.set_xlabel('Speed score')
ax5_2.set_ylabel('Density')

ax5_3 = fig.add_subplot(gs[4, 2])

ratios =  [len(df_py[df_py['speed_score'] > 0.3])/len(df_py), len(df_int[df_int['speed_score']>0.3])/len(df_int)]
colors = ['black', 'red']
ax5_3.bar([0,1],ratios, color=colors)
ax5_3.set_ylabel("% Neurons")
ax5_3.set_xticks([0,1])
ax5_3.set_xticklabels(["Pyramidal","Interneuron"],rotation=-15)

ax5_3.set_yticks([0,0.5,1])
ax5_3.set_yticklabels([0,50,100])

ax5_3.spines['top'].set_visible(False)
ax5_3.spines['right'].set_visible(False)
ax5_3.spines['bottom'].set_visible(True)
ax5_3.spines['left'].set_visible(True)
ax5_3.set_ylim([0,1])

ax5_4 = fig.add_subplot(gs[4, 3])
ax5_5 = fig.add_subplot(gs[4, 4])
idx=0
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
test_hy = ['two-sided','two-sided']
for ax in [ax5_4,ax5_5]:
    # Load Good units

    df_good = df_loaded[df_loaded['unit_quality'] == "good"]
    df_a  = df_good[(df_good['buzaki_cell_type']=='narrow_spike_interneurons') & (df_good['session'] == "A")]

    base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
    control_ids = ['65165', '65091', '63383', '66539', '65622']
    exp_ids = ['65588', '63385', '66538', '66537', '66922']

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

    # Statistical comparisons for scalar metrics
    metrics = ['mean_firing_rate','speed_score']
    titles = ['Firing rate (Hz)', 'Speed score']

    # Define custom colors
    control_color = 'cyan'  # Dark blue for Control
    exp_color = "magenta"  # Light blue for Experimental
    metric=metrics[idx]
    control_values = control_df[metric].dropna()
    exp_values = exp_df[metric].dropna()
    
    if len(control_values) > 0 and len(exp_values) > 0:
        control_mean = control_values.mean()
        exp_mean = exp_values.mean()
        control_sem = control_values.sem()
        exp_sem = exp_values.sem()
        print(f"\nComparison for {metric}:")
        print(f"Control mean: {control_mean:.2f} ± {control_sem:.2f}")
        print(f"Experimental mean: {exp_mean:.2f} ± {exp_sem:.2f}")
        
        # Mann-Whitney U test
        control_array = np.asarray(control_values.values, dtype=float)
        exp_array = np.asarray(exp_values.values, dtype=float)

        # Remove NaN values
        control_clean = control_array[~np.isnan(control_array)]
        exp_clean = exp_array[~np.isnan(exp_array)]

        u_stat, p_val = stats.mannwhitneyu(control_clean, exp_clean, alternative = test_hy[idx])

        # Prepare data for Seaborn plotting
        plot_df = pd.DataFrame({
            'value': pd.concat([control_values, exp_values]),
            'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
        })
        
        # Check if deviation is "too large" (using coefficient of variation > 1 as threshold)
        all_values = plot_df['value']
        cv = all_values.std() / all_values.mean()  # Coefficient of variation
        use_log_scale = cv > 1 and all_values.min() > 0  # Ensure positive values for log scale
        
        # Filter out outliers (e.g., beyond 3 standard deviations)
        mean_val = all_values.mean()
        std_val = all_values.std()
        plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                (plot_df['value'] <= mean_val + 3 * std_val)]
        
        sns.violinplot(
                data=plot_df_filtered, x='group', y='value', ax=ax,inner = "quartiles",
                palette={"Control": control_color, "Experimental": exp_color}, width=0.8, cut=0, edgecolor='white'
            )
        # Add individual points with matching colors
        sns.swarmplot(
                    data=plot_df_filtered, x='group', y='value', ax=ax, size=0.5,
                    hue='group', palette={"Control": "black", "Experimental": "black"},
                    alpha=1, legend=False
                )
        ax.set_ylabel(titles[idx])
        ax.set_xlabel('')
        ax.yaxis.grid(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.spines['left'].set_visible(True)
        ax.set_xticklabels(['CRs +', 'CRs -'], rotation = -30)
        y_max = ax.get_ylim()[1]
        bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
        x_positions = [0, 1]  # Adjusted positions for 'Control' and 'Experimental' groups
        if (p_val < 0.05) & (p_val > 0.01):
            ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                                color='black', lw=1.5)
            ax.text(0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')
        elif (p_val < 0.01) & (p_val > 0.001):
            ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                                color='black', lw=1.5)
            ax.text(0.5, y_max + bar_height * 1.1, f'**', ha='center', va='bottom')
        elif p_val < 0.001:
            ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                                color='black', lw=1.5)
            ax.text(0.5, y_max + bar_height * 1.1, f'***', ha='center', va='bottom')
    idx += 1
    print(idx)


ax5_6 = fig.add_subplot(gs[4, 5])

# Data
groups = ['CRs +', 'CRs -']
counts = [
    [round(78/113,2)*100, round((113-78)/113,2)*100],  # Group 1: Dark blue (71.4%), Light blue (28.6%)
    [round(64/94,2)*100, round((94-64)/94,2)*100],  # Group 2: Dark blue (71.1%), Light blue (28.9%)
]
percentages = [
    [round(78/113,2)*100, round((113-78)/113,2)*100],  # Group 1: Dark blue (71.4%), Light blue (28.6%)
    [round(64/94,2)*100, round((94-64)/94,2)*100],  # Group 2: Dark blue (71.1%), Light blue (28.9%)
]

# Colors
colors = ['cyan', '#C7FDFD']  # For CRs +
colors1 = ['magenta', "#FDC7F8"]    # For CRs -

# Create bar chart
ax = ax5_6 

# Set positions for bars to be closer
bar_width = 0.35  # Reduced width to bring bars closer
x = np.arange(len(groups))  # Create positions for the bars
x = x * 0.5  # Scale x positions to reduce gap between bars

# Plot stacked bars
bars = []
for i in range(len(counts[0])):  # For each category (0 and 1)
    # Create bars for each group with appropriate colors
    bar = ax.bar(x, [counts[j][i] for j in range(len(counts))], 
                 bottom=[sum(counts[j][:i]) for j in range(len(counts))], 
                 color=[colors[i] if j == 0 else colors1[i] for j in range(len(counts))], 
                 label=f'Category {i+1}' if i == 0 else None, 
                 width=bar_width)
    bars.append(bar)

# Add count and percentage labels on top of each segment
for i, bar_group in enumerate(bars):
    if i==0:
        cell_type = "(Speed cell)"
    else:
        cell_type = "(None \nspeed cell)"
    for j, bar in enumerate(bar_group):
        height = bar.get_height()
        total = sum(counts[j])
        bottom = sum(counts[j][:i])
        ax.text(bar.get_x() + bar.get_width()/2, bottom + height/2, 
                f'{percentages[j][i]}%\n {cell_type}', ha='center', va='center', rotation=0, color='black',fontsize=4)

# Customize the plot
ax.set_ylabel('%Neurons')
ax.set_xticks(x)
ax.set_xticklabels(groups, rotation = -30)
ax.legend().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
y_max = ax.get_ylim()[1]
bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
x_positions = x  # Adjusted positions for 'Control' and 'Experimental' groups



units = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_good=df_good[df_good['session'] == "A"]
df_int = df_good[(df_good['buzaki_py_cell_type'] == "narrow_spike_interneurons") & (df_good['session'] == "A")]
df_py = df_good[(df_good['buzaki_py_cell_type'] == "pyramidal") & (df_good['session'] == "A")]

control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
control_df = df_int[df_int['animal_id'].isin(control_ids)]
exp_df = df_int[df_int['animal_id'].isin(exp_ids)]
top_10_indices_exp = exp_df['speed_score'].nlargest(12).index.values
top_10_indices_control = control_df['speed_score'].nlargest(12).index.values

axes = [ax1_1,ax1_2 ,ax1_3 ,ax1_4 ,ax1_5 ,ax1_6, ax2_1,ax2_2 ,ax2_3 ,ax2_4 ,ax2_5,ax2_6 ]
axes1 = [ax3_1,ax3_2 ,ax3_3 ,ax3_4 ,ax3_5 ,ax3_6, ax4_1,ax4_2 ,ax4_3 ,ax4_4 ,ax4_5,ax4_6 ]
i=0
df = control_df.loc[top_10_indices_control]
for index in df.index.values:
    ax=axes[i]
    smoothed_speed= df['speed_collect'].loc[index] # Example velocity array
    smooth_spk = df['speed_fr'].loc[index]
    velocity =smoothed_speed[:len(smooth_spk)] 

    #spike = {'spike_t': [np.random.rand(100) for _ in range(5)]}  # Example spike data
    #positions = np.random.rand(1000)  # Example positions
    #unit_id = np.arange(1, 6)  # Example unit IDs
    # Define bin points and window
    bin_point = np.arange(0, np.nanmax(velocity), 0.04)  # 0 to 0.6 with step 0.04
    window = 0.04 * 1.5
    
    p25, p50, p75 = [], [], []

    # --- Quantile sliding window ---
    for center in bin_point:
        mask = (velocity >= center - window) & (velocity <= center + window)
        qt_spike = smooth_spk[mask]
        if len(qt_spike) > 0:
            q_values = np.quantile(qt_spike, [0.25, 0.5, 0.75])
        else:
            q_values = [np.nan, np.nan, np.nan]
        p25.append(q_values[0])
        p50.append(q_values[1])
        p75.append(q_values[2])

    # --- Subsampling 10% randomly ---
    ten_point = np.random.randint(0, len(velocity), size=int(np.ceil(len(velocity) * 0.1)))

    ax.scatter(
        velocity[ten_point],
        smooth_spk[ten_point],
        s=2,
        edgecolors='white',
        facecolors='cyan',
        linewidths=0.1,
        marker='o',
        alpha=0.15
    )
    ax.plot(bin_point, p50, 'k.', markersize=2)
    ax.plot(bin_point, p25, 'k-')
    ax.plot(bin_point, p75, 'k-')
    ax.yaxis.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    speed_score = df['speed_score'].loc[index]
    y_max = ax.get_ylim()[1]
    bar_height = y_max * 0.1
    ax.text(0.25, y_max * 0.95, f'r = {speed_score:.2f}', ha='center', va='bottom')
    # --- Speed score ---

    ax.set_xlabel("Speed (m/s)")
    ax.set_xticks([0,0.5])
    ax.set_xticklabels([0,25])
    ax.set_ylabel("Firing Rate (Hz)")
    i += 1

i=0
df = exp_df.loc[top_10_indices_exp]
for index in df.index.values:
    ax=axes1[i]
    smoothed_speed= df['speed_collect'].loc[index] # Example velocity array
    smooth_spk = df['speed_fr'].loc[index]
    velocity =smoothed_speed[:len(smooth_spk)] 
    bin_point = np.arange(0, np.nanmax(velocity), 0.04)  # 0 to 0.6 with step 0.04
    window = 0.04 * 1.5
    
    p25, p50, p75 = [], [], []

    # --- Quantile sliding window ---
    for center in bin_point:
        mask = (velocity >= center - window) & (velocity <= center + window)
        qt_spike = smooth_spk[mask]
        if len(qt_spike) > 0:
            q_values = np.quantile(qt_spike, [0.25, 0.5, 0.75])
        else:
            q_values = [np.nan, np.nan, np.nan]
        p25.append(q_values[0])
        p50.append(q_values[1])
        p75.append(q_values[2])

    # --- Subsampling 10% randomly ---
    ten_point = np.random.randint(0, len(velocity), size=int(np.ceil(len(velocity) * 0.1)))

    ax.scatter(
        velocity[ten_point],
        smooth_spk[ten_point],
        s=2,
        edgecolors='white',
        facecolors='magenta',
        linewidths=0.1,
        marker='o',
        alpha=0.15
    )
    ax.plot(bin_point, p50, 'k.', markersize=2)
    ax.plot(bin_point, p25, 'k-')
    ax.plot(bin_point, p75, 'k-')
    ax.yaxis.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    speed_score = df['speed_score'].loc[index]

    y_max = ax.get_ylim()[1]
    bar_height = y_max * 0.1
    ax.text(0.25, y_max * 0.95, f'r = {speed_score:.2f}', ha='center', va='bottom')

    # --- Speed score ---
    ax.set_xlabel("Speed (cm/s)")
    ax.set_xticks([0,0.5])
    ax.set_xticklabels([0,25])
    ax.set_ylabel("Firing Rate (Hz)")
    i += 1

fig.subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95, hspace=1.1, wspace=1.3)
plt.tight_layout()
plt.show()

In [ ]:
len(df_py[df_py['speed_score'] > 0.3])/len(df_py)

In [ ]:
bin_point

In [ ]:
smooth_spk

In [ ]:
units.to_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')